# Baemin_배달의 민족


In [42]:
import simpy
import random

RANDOM_SEED = 42
NUM_COOKS = 2
NUM_DELIVERY_STAFF = 3
SIM_TIME = 120

FOOD_MENU = {'김밥': 2500, '순대': 3000, '라면': 3500, '떡볶이': 4000}


class Customer:
    def __init__(self, env, restaurant, delivery):
        self.env = env
        self.restaurant = restaurant
        self.delivery = delivery
        self.items = []
        self.num_items = 0
        self.price = 0

    def order_food(self):
        # 고객이 주문할 음식의 종류와 개수를 랜덤으로 생성
        for item, price in FOOD_MENU.items():
            if random.random() > 0.3:
                self.items.append(item)
        self.num_items = len(self.items)
        self.price = self.restaurant.calculate_price(self.items)
        print(f'{round(self.env.now)}분: 고객이 주문을 완료했습니다. 주문내역 - {self.items}, 총 {self.price}원입니다.')
        
        # 주문 정보를 로그에 남기고, 요리를 시작
        self.restaurant.food_orders.append((self.items, num_items))  # 주문을 리스트에 추가        
        self.restaurant.cook_food()

        self.delivery.add_to_delivery_queue(self)
        yield self.env.timeout(0)

    def receive_food(self):
        print(f'{round(self.env.now)}분: 고객이 음식을 받았습니다.')
        yield self.env.timeout(0)

        
class Restaurant:
    def __init__(self, env, num_cooks):
        self.env = env
        self.num_cooks = num_cooks
        self.food_orders = simpy.Store(env)
        self.cooking_time = 5
        
        for i in range(self.num_cooks):
            self.env.process(self.cook_food())

    def cook_food(self):
        while True:
            order = yield self.food_orders.get()
            yield self.env.timeout(self.cooking_time)
            print(f'{round(self.env.now)}분: {order.num_items}개의 {order.items}를 조리했습니다.')

    def calculate_price(self, items):
        return sum([FOOD_MENU[item] for item in items])


class Delivery:
    def __init__(self, env, num_delivery_staff):
        self.env = env
        self.delivery_queue = simpy.Store(env)
        self.delivery_time = 10
        self.num_delivery_staff = num_delivery_staff
        
        for i in range(self.num_delivery_staff):
            self.env.process(self.deliver_food())

    def add_to_delivery_queue(self, customer):
        self.delivery_queue.put(customer)

    def deliver_food(self):
        while True:
            customer = yield self.delivery_queue.get()
            yield self.env.timeout(self.delivery_time)
            print(f'{round(self.env.now)}분: 고객에게 음식을 배달했습니다.')
            self.env.process(customer.receive_food())


def customer_arrivals(env, num_customers, restaurant, delivery):
    for i in range(num_customers):
        customer = Customer(env, restaurant, delivery)
        env.process(customer.order_food())
        inter_arrival_time = random.expovariate(1/5)
        yield env.timeout(inter_arrival_time)


# 시뮬레이션 함수 정의
def run_simulation(env, num_customers, num_cooks, num_delivery_staff):
    # 식당, 배달 객체 생성
    restaurant = Restaurant(env, num_cooks)
    delivery = Delivery(env, num_delivery_staff)

    # 고객 도착
    for i in range(num_customers):
        customer = Customer(env, restaurant, delivery)
        env.process(customer.order_food())
        # 고객 도착 간격
        yield env.timeout(random.randint(1, 5))


In [43]:
# 시뮬레이션 실행
env = simpy.Environment()
env.process(run_simulation(env, NUM_CUSTOMERS, NUM_COOKS, NUM_DELIVERY_STAFF))
env.run(until=SIM_TIME)

0분: 고객이 주문을 완료했습니다. 주문내역 - ['김밥', '라면', '떡볶이'], 총 10000원입니다.


AttributeError: 'Store' object has no attribute 'append'